In [ ]:
## Only Run Once

import pandas as pd
from sas7bdat import SAS7BDAT

#summ = pd.read_sas("E:DS\Funds\MF_Fund Summary.sas7bdat")

with SAS7BDAT("E:DS\Funds\MF_Fund Summary.sas7bdat") as f:
    df = f.to_data_frame()
    "E:DS\Funds\MF_DailyReturns.sas7bdat"
    "E:DS\Funds\MF_FrontLoad.sas7bdat"
    "E:DS\Funds\MF_FundDividends.sas7bdat"
    "E:DS\Funds\MF_FundPortMap.sas7bdat"
    "E:DS\Funds\MF_Monthly Returns.sas7bdat"
    "E:DS\Funds\MF_RearLoad.sas7bdat"
    

with SAS7BDAT("E:DS\Funds\MF_DailyReturns.sas7bdat") as f:
    dret = f.to_data_frame()

with SAS7BDAT("E:DS\Funds\MF_FrontLoad.sas7bdat") as f:
    frol = f.to_data_frame()

with SAS7BDAT("E:DS\Funds\MF_FundDividends.sas7bdat") as f:
    fudi = f.to_data_frame()

with SAS7BDAT("E:DS\Funds\MF_FundPortMap.sas7bdat") as f:
    fupo = f.to_data_frame()

with SAS7BDAT("E:DS\Funds\MF_Monthly Returns.sas7bdat") as f:
    moret = f.to_data_frame()

with SAS7BDAT("E:DS\Funds\MF_RearLoad.sas7bdat") as f:
    mfrer = f.to_data_frame()

import _pickle as pickle

pickle.dump(df, open("E:DS\Funds\summ.p", "wb"))

pickle.dump(dret, open("E:DS\Funds\dret.p", "wb"))

frol.head()

frol.to_hdf("E:DS\Funds\\frol.h5", 'key_to_store', table=True, mode='w')

#pickle.dump(frol, open("E:DS\Funds\frol.p", "wb"))

#pickle.dump(fudi, open("E:DS\Funds\fudi.p", "wb"))
fudi.to_hdf("E:DS\Funds\\fudi.h5", 'key_to_store', table=True, mode='w')

#pickle.dump(fudi, open("E:DS\Funds\fupo.p", "wb"))
fupo.to_hdf("E:DS\Funds\\fupo.h5", 'key_to_store', table=True, mode='w')

pickle.dump(moret, open("E:DS\Funds\moret.p", "wb"))
pickle.dump(mfrer, open("E:DS\Funds\mfrer.p", "wb"))

In [1]:
import pandas as pd
from sklearn import preprocessing
import _pickle as pickle

from datetime import timedelta

start = 19600101

def datecalc(mfrer,start =19600101 ):
    mfrer["BEGDAT"] = start

    mfrer["BEGDAT"] = pd.to_datetime(mfrer['BEGDAT'].astype(str), format='%Y%m%d')

    mfrer["BEGDT"] = mfrer["BEGDT"].apply(lambda x: timedelta(days = int(x)))

    mfrer["BEGDAT"] = mfrer["BEGDAT"] + mfrer["BEGDT"]

    mfrer["ENDDAT"] = start

    mfrer["ENDDAT"] = pd.to_datetime(mfrer['ENDDAT'].astype(str), format='%Y%m%d')

    mfrer["ENDDT"] = mfrer["ENDDT"].apply(lambda x: timedelta(days = int(x)))

    mfrer["ENDDAT"] = mfrer["ENDDAT"] + mfrer["ENDDT"]
    return mfrer

def datec(mfrer,start =19600101 ):
    mfrer["CALDAT"] = start

    mfrer["CALDAT"] = pd.to_datetime(mfrer['CALDAT'].astype(str), format='%Y%m%d')

    mfrer["CALDT_T"] = mfrer["CALDT"].apply(lambda x: timedelta(days = int(x)))

    mfrer["CALDAT"] = mfrer["CALDAT"] + mfrer["CALDT_T"]
    return mfrer

pd.options.display.max_columns = None

summ = pickle.load(open("E:DS\Funds\summ.p", "rb"))
# Select AQ
summ = summ[summ["SUMMARY_PERIOD2"]=="AQ"]
summ = datec(summ)


summ[["FIRST_OFFER_DT", "MGR_DT", "END_DT", "UNREALIZED_APP_DT","FISCAL_YEAREND"]] = summ[["FIRST_OFFER_DT", "MGR_DT", "END_DT", "UNREALIZED_APP_DT","FISCAL_YEAREND"]].fillna(method="ffill").fillna(method="bfill")


summ[["FIRST_OFFER_DT", "MGR_DT", "END_DT", "UNREALIZED_APP_DT","FISCAL_YEAREND"]] = summ[["FIRST_OFFER_DT", "MGR_DT", "END_DT", "UNREALIZED_APP_DT","FISCAL_YEAREND"]].fillna(method="ffill").fillna(method="bfill")

summ["FISCAL_YEAREND_A"] = start

summ["FISCAL_YEAREND_A"] = pd.to_datetime(summ['FISCAL_YEAREND_A'].astype(str), format='%Y%m%d')

summ["FISCAL_YEAREND_T"] = summ["FISCAL_YEAREND"].apply(lambda x: timedelta(days = int(x)))


summ["FISCAL_YEAREND_A"] = summ["FISCAL_YEAREND_A"] + summ["FISCAL_YEAREND_T"]

#sum = datesumm(summ)

del summ["CALDT_T"], summ["FISCAL_YEAREND_T"]

## Fill Blank with EMPT
summ = summ.applymap(lambda x: "EMPT" if x=="" else x)

# Decrease cardinality for 4 variables
mg_co = list(summ["MGMT_CD"].value_counts().head(20).index.values)
summ["MGMT_CD"] = summ["MGMT_CD"].apply(lambda x: x if x in mg_co else "OTHER")

mg_co = list(summ["MGR_NAME"].value_counts().head(20).index.values)
summ["MGR_NAME"] = summ["MGR_NAME"].apply(lambda x: x if x in mg_co else "OTHER")

mg_co = list(summ["CRSP_OBJ_CD"].value_counts().head(50).index.values)
summ["CRSP_OBJ_CD"] = summ["CRSP_OBJ_CD"].apply(lambda x: x if x in mg_co else "OTHER")

mg_co = list(summ["LIPPER_OBJ_CD"].value_counts().head(40).index.values)
summ["LIPPER_OBJ_CD"] = summ["LIPPER_OBJ_CD"].apply(lambda x: x if x in mg_co else "OTHER")

dates = ["FIRST_OFFER_DT", "MGR_DT", "END_DT", "UNREALIZED_APP_DT", "NAV_52W_L_DT", "NAV_52W_H_DT", "TNA_LATEST_DT","NAV_LATEST_DT"]

# Filna 0 makes more sense here.
for d in dates:
    summ[d] = summ[d].fillna(value=0)
    summ["CALDT_M_"+d] =summ["CALDT"].astype(int) - summ[d].astype(int)
    del summ[d]
    

os_bin = ["OPEN_TO_INV", "SALES_RESTRICT","INDEX_FUND_FLAG", "ACCRUAL_FUND", "RETAIL_FUND", "INST_FUND", "M_FUND", "VAU_FUND", "ET_FLAG"]

os_df = pd.get_dummies(summ[os_bin])
summ.drop(os_bin, axis=1,inplace=True)
summ = pd.concat((summ,os_df),axis=1)

rem_list = ["SUMMARY_PERIOD2","ADV_NAME", "CRSP_PORTNO", "CUSIP8", "POLICY", "SI_OBJ_CD", "WBRGER_OBJ_CD", "DEAD_FLAG", "DELIST_CD", "FUND_NAME", "TICKER", "NCUSIP", "CRSP_CL_GRP", "MGMT_NAME"]

summ.drop(rem_list, axis=1, inplace=True)


le = preprocessing.LabelEncoder()

for col in summ:
    if summ[col].dtype == 'object':
        summ[col] = le.fit_transform(summ[col])

summ.shape

(473589, 79)

In [2]:

moret = pickle.load(open("E:DS\Funds\moret.p", "rb"))
mfrer = pickle.load(open("E:DS\Funds\mfrer.p", "rb"))


mfrer["BEGDAT"] = start


mfrer = datecalc(mfrer)

#moret = datecalc(moret)



In [3]:

mfrer = mfrer[~mfrer["REAR_LOAD"].isin([-99.0,0.00])]

mfrer = mfrer.replace(-99.0, 0)

mfrer = mfrer.drop(["BEGDT", "ENDDT", "REAR_GROUP_NO", "INV_LVL"],axis=1)

mfgo = mfrer.groupby(["BEGDAT","ENDDAT","LOAD_TYPE","CRSP_FUNDNO"]).agg(["sum","mean"]).reset_index()


mfgo.columns = ['BEGDAT', 'ENDDAT', 'LOAD_TYPE', 'CRSP_FUNDNO', 'REAR_LOAD_sum',
       'REAR_LOAD_mean', 'TIME_PERIOD_sum', 'TIME_PERIOD_mean']


mfrer["LOAD_TYPE"].value_counts()

mfgor = mfgo[mfgo["LOAD_TYPE"]=="R"].reset_index(drop=True)
mfgoc = mfgo[mfgo["LOAD_TYPE"]=="C"].reset_index(drop=True)

In [7]:
dret = pickle.load(open("E:DS\Funds\dret_1.p", "rb"))
frol = pd.read_hdf("E:DS\Funds\\frol.h5")

In [8]:
frol = datecalc(frol)
#dret = datec(dret)

In [9]:
#pickle.dump(dret, open("E:DS\Funds\dret_1.p", "wb"))

In [10]:
frol = frol.replace(-99.0,0)

frol = frol.replace(999999999.0,0)

frol["front_loaded"] = frol["FRONT_LOAD"]*frol["DOLLAR_AMT"]

frol = frol.drop("FRONT_GROUP_NO",axis=1)


frolg = frol.groupby(["BEGDAT","ENDDAT","CRSP_FUNDNO"]).agg(["sum","mean"]).reset_index()
frolg.columns = ['BEGDAT','ENDDAT','CRSP_FUNDNO','FRONT_LOAD_sum','FRONT_LOAD_mean', 'DOLLAR_AMT_sum','DOLLAR_AMT_mean', 'front_loaded_sum', 'front_loaded_mean']

frolg["DIFF_DATES"] =  (frolg["BEGDAT"] - frolg["ENDDAT"]).dt.days
frolg  = frolg.drop("ENDDAT",axis=1).groupby(["CRSP_FUNDNO","BEGDAT"]).mean().reset_index()

dret_n = pd.merge(dret, frolg, left_on=["CRSP_FUNDNO","CALDAT"],right_on=["CRSP_FUNDNO","BEGDAT"], how="left")


In [56]:
frol.head()

,CRSP_FUNDNO,BEGDT,ENDDT,FRONT_LOAD,DOLLAR_AMT,BEGDAT,ENDDAT,front_loaded
0,1.0,13879 days,14122 days,0.000,0.0,1997-12-31,1998-08-31,0.0
1,2.0,13879 days,14122 days,0.000,0.0,1997-12-31,1998-08-31,0.0
2,3.0,12166 days,14853 days,0.045,100000.0,1993-04-23,2000-08-31,4500.0
3,3.0,12166 days,14853 days,0.035,250000.0,1993-04-23,2000-08-31,8750.0
4,3.0,12166 days,14853 days,0.030,500000.0,1993-04-23,2000-08-31,15000.0


In [11]:
mfgoc.columns = ['BEGDAT', 'ENDDAT', 'LOAD_TYPE', 'CRSP_FUNDNO', 'REAR_LOAD_sum_c',
       'REAR_LOAD_mean_c', 'TIME_PERIOD_sum_c', 'TIME_PERIOD_mean_c']

In [12]:
mfgor["DIFF_DATES"] =  (mfgor["BEGDAT"] - mfgor["ENDDAT"]).dt.days
mfgor  = mfgor.drop("ENDDAT",axis=1).groupby(["CRSP_FUNDNO","BEGDAT"]).mean().reset_index()

dret_n = pd.merge(dret_n, mfgor, left_on=["CRSP_FUNDNO","CALDAT"],right_on=["CRSP_FUNDNO","BEGDAT"], how="left")

In [13]:
mfgoc["DIFF_DATES"] =  (mfgoc["BEGDAT"] - mfgoc["ENDDAT"]).dt.days
mfgoc  = mfgoc.drop("ENDDAT",axis=1).groupby(["CRSP_FUNDNO","BEGDAT"]).mean().reset_index()

dret_n = pd.merge(dret_n, mfgoc, left_on=["CRSP_FUNDNO","CALDAT"],right_on=["CRSP_FUNDNO","BEGDAT"], how="left")

In [14]:
#dret_n = dret_n.fillna(value=0)
dret_n = dret_n.groupby('CRSP_FUNDNO', sort=False).ffill().bfill()

In [15]:
fudi= pd.read_hdf("E:DS\Funds\\fudi.h5")
fudi = datec(fudi)

mg_co = list(fudi["DIS_TYPE"].value_counts().head(5).index.values)
fudi["DIS_TYPE"] = fudi["DIS_TYPE"].apply(lambda x: x if x in mg_co else "OTHER")

os_df = pd.get_dummies(fudi["DIS_TYPE"])
fudi.drop("DIS_TYPE", axis=1,inplace=True)
fudi = pd.concat((fudi,os_df),axis=1)

fudi["SPL_RATIO"] = fudi["SPL_RATIO"].fillna(value=0)

# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# fudi["DIS_TYPE"] = le.fit_transform(fudi["DIS_TYPE"])

fudi = fudi.sort_values(["CRSP_FUNDNO","CALDAT"]).reset_index(drop=True)

for d in ["DIS_AMT", "REINVEST_NAV", "SPL_RATIO"]:
    for c in ["CL","CS","D","DT","DW","OTHER"]:
        fudi[d+"_x_"+c] = fudi[d]*fudi[c]
        

fudi['year'] = pd.DatetimeIndex(fudi['CALDAT']).year
fudi['month'] = pd.DatetimeIndex(fudi['CALDAT']).month
fudi["year_month"] = fudi['year'].astype(str) + fudi['month'].astype(str)

del fudi["year"], fudi["month"]

fudga = fudi.drop("CALDT",axis=1).groupby(["CRSP_FUNDNO","year_month"]).agg(["sum","mean"]).reset_index()

lv1 = ['DIS_AMT', 'REINVEST_NAV', 'SPL_RATIO', 'CL', 'CS', 'D', 'DT', 'DW', 'OTHER', 'DIS_AMT_x_CL', 'DIS_AMT_x_CS', 'DIS_AMT_x_D', 'DIS_AMT_x_DT', 'DIS_AMT_x_DW', 'DIS_AMT_x_OTHER', 'REINVEST_NAV_x_CL', 'REINVEST_NAV_x_CS', 'REINVEST_NAV_x_D', 'REINVEST_NAV_x_DT', 'REINVEST_NAV_x_DW', 'REINVEST_NAV_x_OTHER', 'SPL_RATIO_x_CL', 'SPL_RATIO_x_CS', 'SPL_RATIO_x_D', 'SPL_RATIO_x_DT', 'SPL_RATIO_x_DW', 'SPL_RATIO_x_OTHER']
lv2 = ['sum', 'mean']
cols = ["CRSP_FUNDNO", "year_month"]
for one in lv1:
    for two in lv2:
        cols.append(one+"_"+two)


fudga.columns = cols

fudga.head(10)

,CRSP_FUNDNO,year_month,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean
0,1.0,199710,0.0886,0.08860,15.24,15.240,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0886,0.08860,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,15.24,15.24,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,199711,0.0859,0.08590,15.26,15.260,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0859,0.08590,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,15.26,15.26,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,199712,0.1087,0.05435,30.61,15.305,0.0,0.0,0,0.0,0,0.0,1,0.5,0,0.0,0,0.0,1,0.5,0.0,0.0,0.0,0.0,0.0887,0.04435,0.0,0.0,0.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0,15.30,7.65,0.0,0.0,0.0,0.0,15.31,7.655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,19972,0.0785,0.07850,14.98,14.980,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0785,0.07850,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,14.98,14.98,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,19973,0.0834,0.08340,14.72,14.720,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0834,0.08340,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,14.72,14.72,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,19974,0.0839,0.08390,14.76,14.760,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0839,0.08390,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,14.76,14.76,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,19975,0.0896,0.08960,14.87,14.870,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0896,0.08960,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,14.87,14.87,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,19976,0.0859,0.08590,14.97,14.970,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0859,0.08590,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,14.97,14.97,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,19977,0.0890,0.08900,15.32,15.320,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0890,0.08900,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,15.32,15.32,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,19978,0.0895,0.08950,15.07,15.070,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0895,0.08950,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,15.07,15.07,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
summ['NEW_DATE'] = summ['CALDAT'].apply(lambda x: x - pd.DateOffset(years=1))

summ["Flag_ex"] = 0

summ['year'] = pd.DatetimeIndex(summ['CALDAT']).year
summ['month'] = pd.DatetimeIndex(summ['CALDAT']).month
summ["year_month"] = summ['year'].astype(str) + summ['month'].astype(str)

summ['year_1'] = pd.DatetimeIndex(summ['NEW_DATE']).year
summ['month_1'] = pd.DatetimeIndex(summ['NEW_DATE']).month
summ["year_month_1"] = summ['year_1'].astype(str) + summ['month_1'].astype(str)

del summ["year"], summ["month"]
del summ["year_1"], summ["month_1"]
summ.head()

fudit = pd.merge(fudga, summ[["Flag_ex","CRSP_FUNDNO","year_month_1"]],left_on=["CRSP_FUNDNO","year_month"],right_on=["CRSP_FUNDNO","year_month_1"],how="left")

fudit = fudit[fudit["CRSP_FUNDNO"].isin(list(summ["CRSP_FUNDNO"].unique()))]

summ["Flag"] = summ["CALDT"].astype(str) +"_" + summ["CRSP_FUNDNO"].astype(str)

fudit = pd.merge(fudit, summ[["Flag","CRSP_FUNDNO","year_month"]], on=["CRSP_FUNDNO","year_month"],how="left")


fudit[["Flag","Flag_ex"]] = fudit[["Flag","Flag_ex"]].fillna(value="")
fudit["Flag_final"] = fudit["Flag"] + fudit["Flag_ex"].astype(str)
fudit["Flag_final"] = fudit["Flag_final"].apply(lambda x: None if x=="" else x)


fudit = fudit.sort_values(["CRSP_FUNDNO","year_month"])
fudit["Flag_final"] = fudit.groupby('CRSP_FUNDNO', sort=False)['Flag_final'].apply(lambda x: x.bfill())

#pd.options.display.max_rows = None

fudit[["CRSP_FUNDNO","year_month","Flag_final","DIS_AMT_x_D_sum"]].tail(100)

fudita = fudit[~fudit["Flag_final"].isin([None,'0.0'])]
#fudita = fudita[fudit["Flag"]]

fudita = fudita.groupby(["Flag_final"], sort=False).tail(12)

fudita = fudita.groupby(["CRSP_FUNDNO","year_month"]).mean()

fudita = fudita.reset_index()

summ_n = pd.merge(summ, fudita, on=["CRSP_FUNDNO","year_month"],how="left")

summ_n.head()

,CRSP_FUNDNO,CALDT,MGMT_CD,MGR_NAME,MERGE_FUNDNO,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,FISCAL_YEAREND_A,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,NEW_DATE,Flag_ex,year_month,year_month_1,Flag,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean
0,1.0,14609.0,15,15,8441.0,13.97,200.1,0.070372,15.16,13.97,0.983104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,12,40,2,49,3,2,-99.0000,NaN,0.0042,0.084,0.64,14517.0,1999-12-31,1999-09-30,1063,729,-213,0,0,336,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_1.0,0.083648,0.083648,13.97,13.970000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.083648,0.083648,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000,13.97,13.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,14609.0,15,15,8425.0,18.90,140.7,0.030688,20.11,17.39,0.580000,0.710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,151,171,12,111,1,2,-99.0000,NaN,0.0000,0.000,0.32,14517.0,1999-12-31,1999-09-30,1061,1094,-244,0,303,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_2.0,1.290000,0.645000,37.70,18.850000,0.0,0.0,1.0,0.500000,0.0,0.000000,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,0.0,0.710,0.355000,0.000,0.000000,0.0,0.0,0.580,0.290000,0.000000,0.000000,0.0,0.0,18.85,9.425000,0.00,0.000000,0.0,0.0,18.85,9.425000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,14609.0,15,15,NaN,9.66,0.1,0.000104,10.93,9.40,0.001000,0.095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,105,116,34,146,3,2,0.0025,0.0025,0.0200,1.175,0.40,14609.0,1999-12-31,1999-12-31,2440,1094,-244,0,172,351,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_3.0,0.069000,0.023000,28.90,9.633333,0.0,0.0,1.0,0.333333,1.0,0.333333,0.0,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.034,0.011333,0.034,0.011333,0.0,0.0,0.001,0.000333,0.000000,0.000000,0.0,0.0,9.62,3.206667,9.62,3.206667,0.0,0.0,9.66,3.220000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,14609.0,15,15,8513.0,23.70,45.7,0.005274,23.70,16.35,0.125000,0.735,Na

In [20]:
#summ_n.to_hdf("E:DS\Funds\summ_n.h5", 'key_to_store', table=True, mode='w')


In [21]:
#dret_n.to_hdf("E:DS\Funds\dret_n.h5", 'key_to_store', table=True, mode='w')

In [25]:
def subtract_years(dt, years):
    try:
        dt = dt.replace(year=dt.year-years)
    except ValueError:
        dt = dt.replace(year=dt.year-years, day=dt.day-1)
    return dt

In [67]:
## for eah calc return and return for next year.
from dateutil.relativedelta import relativedelta

dret_n['OLD_DATE'] = dret_n['CALDAT'] - timedelta(days=365)

dret_n["FUTURE_DATE"]= dret_n['CALDAT'] + timedelta(days=365)

dret_n.head()

,CRSP_FUNDNO,CALDT,DRET,DNAV_x,CALDAT,CALDT_T,BEGDAT_x,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,BEGDAT_y,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,BEGDAT,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,OLD_DATE_x,OLD_DATE_y,DNAV_y,OLD_DATE,FUTURE_DATE
0,1.0,14123.0,-0.001987,15.07,1998-09-01,14123 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.02,0.02,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1997-09-01,1998-09-01,14.14,1997-09-01,1999-09-01
1,1.0,14124.0,0.001327,15.09,1998-09-02,14124 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.02,0.02,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1997-09-02,1998-09-02,14.11,1997-09-02,1999-09-02
2,1.0,14125.0,0.000663,15.10,1998-09-03,14125 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.02,0.02,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1997-09-03,1998-09-03,14.18,1997-09-03,1999-09-03
3,1.0,14126.0,0.001325,15.12,1998-09-04,14126 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.02,0.02,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1997-09-04,NaT,NaN,1997-09-04,1999-09-04
4,1.0,14130.0,-0.000661,15.11,1998-09-08,14130 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.02,0.02,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1997-09-08,1998-09-08,14.16,1997-09-08,1999-09-08


In [32]:
dret_n = pd.merge(dret_n, dret_n[["CRSP_FUNDNO","OLD_DATE","DNAV"]], left_on=["CRSP_FUNDNO","CALDAT"], right_on=["CRSP_FUNDNO","OLD_DATE"], how="left")

In [69]:
dret_n["DNAV_x"].head()

0    15.07
1    15.09
2    15.10
3    15.12
4    15.11
Name: DNAV_x, dtype: float64

In [71]:
dret_n = pd.merge(dret_n, dret_n[["CRSP_FUNDNO","FUTURE_DATE","DNAV_x"]], left_on=["CRSP_FUNDNO","CALDAT"], right_on=["CRSP_FUNDNO","FUTURE_DATE"], how="left")

In [36]:
summ_n.head()

,CRSP_FUNDNO,CALDT,MGMT_CD,MGR_NAME,MERGE_FUNDNO,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,FISCAL_YEAREND_A,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,NEW_DATE,Flag_ex,year_month,year_month_1,Flag,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean
0,1.0,14609.0,15,15,8441.0,13.97,200.1,0.070372,15.16,13.97,0.983104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,12,40,2,49,3,2,-99.0000,NaN,0.0042,0.084,0.64,14517.0,1999-12-31,1999-09-30,1063,729,-213,0,0,336,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_1.0,0.083648,0.083648,13.97,13.970000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.083648,0.083648,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000,13.97,13.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,14609.0,15,15,8425.0,18.90,140.7,0.030688,20.11,17.39,0.580000,0.710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,151,171,12,111,1,2,-99.0000,NaN,0.0000,0.000,0.32,14517.0,1999-12-31,1999-09-30,1061,1094,-244,0,303,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_2.0,1.290000,0.645000,37.70,18.850000,0.0,0.0,1.0,0.500000,0.0,0.000000,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,0.0,0.710,0.355000,0.000,0.000000,0.0,0.0,0.580,0.290000,0.000000,0.000000,0.0,0.0,18.85,9.425000,0.00,0.000000,0.0,0.0,18.85,9.425000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,14609.0,15,15,NaN,9.66,0.1,0.000104,10.93,9.40,0.001000,0.095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,105,116,34,146,3,2,0.0025,0.0025,0.0200,1.175,0.40,14609.0,1999-12-31,1999-12-31,2440,1094,-244,0,172,351,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_3.0,0.069000,0.023000,28.90,9.633333,0.0,0.0,1.0,0.333333,1.0,0.333333,0.0,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.034,0.011333,0.034,0.011333,0.0,0.0,0.001,0.000333,0.000000,0.000000,0.0,0.0,9.62,3.206667,9.62,3.206667,0.0,0.0,9.66,3.220000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,14609.0,15,15,8513.0,23.70,45.7,0.005274,23.70,16.35,0.125000,0.735,Na

In [42]:
dret_n["DNAV_y"].isnull().sum()

37441696

In [75]:
dret_n["DNAV_y"].isnull().sum()

37441696

In [72]:
dret_n.tail()

,CRSP_FUNDNO,CALDT,DRET,DNAV_x_x,CALDAT,CALDT_T,BEGDAT_x,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,BEGDAT_y,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,BEGDAT,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,OLD_DATE_x,OLD_DATE_y,DNAV_y,OLD_DATE,FUTURE_DATE_x,FUTURE_DATE_y,DNAV_x_y
121729483,93365.0,21266.0,-0.017037,13.27,2018-03-23,21266 days,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2017-03-23,NaT,NaN,2017-03-23,2019-03-23,NaT,NaN
121729484,93365.0,21269.0,0.029390,13.66,2018-03-26,21269 days,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2017-03-26,NaT,NaN,2017-03-26,2019-03-26,NaT,NaN
121729485,93365.0,21270.0,-0.019034,13.40,2018-03-27,21270 days,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2017-03-27,NaT,NaN,2017-03-27,2019-03-27,NaT,NaN
121729486,93365.0,21271.0,-0.000746,13.39,2018-03-28,21271 days,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2017-03-28,NaT,NaN,2017-03-28,2019-03-28,NaT,NaN
121729487,93365.0,21272.0,0.014937,13.59,2018-03-29,21272 days,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2017-03-29,NaT,NaN,2017-03-29,2019-03-29,NaT,NaN


In [99]:
summ_f = pd.merge(summ_n, dret_n, on=["CRSP_FUNDNO","CALDAT"],how="left")

In [77]:
summ_f.head()

,CRSP_FUNDNO,CALDT_x,MGMT_CD,MGR_NAME,MERGE_FUNDNO,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,FISCAL_YEAREND_A,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,NEW_DATE,Flag_ex,year_month,year_month_1,Flag,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean,CALDT_y,DRET,DNAV_x_x,CALDT_T,BEGDAT_x,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,BEGDAT_y,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,BEGDAT,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,OLD_DATE_x,OLD_DATE_y,DNAV_y,OLD_DATE,FUTURE_DATE_x,FUTURE_DATE_y,DNAV_x_y
0,1.0,14609.0,15,15,8441.0,13.97,200.1,0.070372,15.16,13.97,0.983104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,12,40,2,49,3,2,-99.0000,NaN,0.0042,0.084,0.64,14517.0,1999-12-31,1999-09-30,1063,729,-213,0,0,336,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_1.0,0.083648,0.083648,13.97,13.970000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.083648,0.083648,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000,13.97,13.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14609.0,0.003832,13.97,14609 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.020,0.020,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1998-12-31,NaT,NaN,1998-12-31,2000-12-30,1999-12-31,15.07
1,2.0,14609.0,15,15,8425.0,18.90,140.7,0.030688,20.11,17.39,0.580000,0.710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,151,171,12,111,1,2,-99.0000,NaN,0.0000,0.000,0.32,14517.0,1999-12-31,1999-09-30,1061,1094,-244,0,303,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_2.0,1.290000,0.645000,37.70,18.850000,0.0,0.0,1.0,0.500000,0.0,0.000000,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,0.0,0.710,0.355000,0.000,0.000000,0.0,0.0,0.580,0.290000,0.000000,0.000000,0.0,0.0,18.85,9.425000,0.00,0.000000,0.0,0.0,18.85,9.425000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14609.0,0.002653,18.90,14609 days,2000-03-31,0.185,0.030833,1.001

In [80]:
#~summ_f["DNAV_x"].isnull()

In [78]:
#summ_d = summ_f[~summ_f["DNAV_y"].isnull()].reset_index(drop=True)

In [85]:
#summ_d = summ_f[~summ_f["DNAV_x_y"].isnull()].reset_index(drop=True)

In [101]:
summ_f.head()

,CRSP_FUNDNO,CALDT_x,MGMT_CD,MGR_NAME,MERGE_FUNDNO,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,FISCAL_YEAREND_A,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,NEW_DATE,Flag_ex,year_month,year_month_1,Flag,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean,CALDT_y,DRET,DNAV_x_x,CALDT_T,BEGDAT_x,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,BEGDAT_y,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,BEGDAT,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,OLD_DATE_x,OLD_DATE_y,DNAV_y,OLD_DATE,FUTURE_DATE_x,FUTURE_DATE_y,DNAV_x_y
0,1.0,14609.0,15,15,8441.0,13.97,200.1,0.070372,15.16,13.97,0.983104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,12,40,2,49,3,2,-99.0000,NaN,0.0042,0.084,0.64,14517.0,1999-12-31,1999-09-30,1063,729,-213,0,0,336,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_1.0,0.083648,0.083648,13.97,13.970000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.083648,0.083648,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000,13.97,13.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14609.0,0.003832,13.97,14609 days,2000-03-31,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,2000-05-01,0.020,0.020,3.0,3.0,-122.0,2004-07-01,0.01,0.01,12.0,12.0,-5021.0,1998-12-31,NaT,NaN,1998-12-31,2000-12-30,1999-12-31,15.07
1,2.0,14609.0,15,15,8425.0,18.90,140.7,0.030688,20.11,17.39,0.580000,0.710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,151,171,12,111,1,2,-99.0000,NaN,0.0000,0.000,0.32,14517.0,1999-12-31,1999-09-30,1061,1094,-244,0,303,2,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1998-12-31,0,199912,199812,14609.0_2.0,1.290000,0.645000,37.70,18.850000,0.0,0.0,1.0,0.500000,0.0,0.000000,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,0.0,0.710,0.355000,0.000,0.000000,0.0,0.0,0.580,0.290000,0.000000,0.000000,0.0,0.0,18.85,9.425000,0.00,0.000000,0.0,0.0,18.85,9.425000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14609.0,0.002653,18.90,14609 days,2000-03-31,0.185,0.030833,1.001

In [107]:
summ_f[["DNAV_x_x","DNAV_x_y","DNAV_y"]].isnull().sum()

DNAV_x_x      4473
DNAV_x_y    150447
DNAV_y      233171
dtype: int64

In [106]:
summ_f[["DNAV_x_x","DNAV_x_y","DNAV_y"]].head()

,DNAV_x_x,DNAV_x_y,DNAV_y
0,13.97,15.07,NaN
1,18.90,17.94,NaN
2,9.66,10.80,NaN
3,23.70,18.13,NaN
4,19.52,19.98,NaN


In [109]:
summ_f["annual_ret_f"] = (summ_f["DNAV_x_x"]-summ_f["DNAV_x_y"])/summ_f["DNAV_x_y"]

In [110]:
summ_f["annual_ret"] = (summ_f["DNAV_x_x"]-summ_f["DNAV_y"])/summ_f["DNAV_y"]

In [111]:
summ_f = summ_f[~summ_f["annual_ret_f"].isnull()]
summ_f = summ_f[~summ_f["annual_ret"].isnull()]

In [112]:
summ_f = summ_f.drop(["OLD_DATE", "FUTURE_DATE_x", "FUTURE_DATE_y","CALDT_x","MERGE_FUNDNO", "FISCAL_YEAREND_A","NEW_DATE", "Flag_ex", "year_month", "year_month_1", "Flag", "CALDT_y", "DRET", "DNAV_x_y","DNAV_x_x", "CALDT_T", "BEGDAT", "BEGDAT_x", "BEGDAT_y", "OLD_DATE_x", "OLD_DATE_y", "DNAV_y"],axis=1)

In [113]:
#Its a better outcome than that with the 9s/
summ_f.head()

,CRSP_FUNDNO,MGMT_CD,MGR_NAME,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,annual_ret_f,annual_ret
45,105.0,15,15,7.16,35.5,NaN,10.09,6.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,61,73,13,106,0,0,0.0100,-99.00,0.0276,0.998,0.57,15491.0,2002-12-31,6409,6573,-5567,278,84,358,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,0.002,0.002,99999.0,99999.0,-519.0,0.01,0.01,12.0,12.0,-5021.0,-0.272358,-0.301463
46,105.0,15,15,10.25,51.7,NaN,10.25,6.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,61,73,13,106,0,0,0.0100,-99.00,0.0294,0.956,0.97,15856.0,2003-12-31,6774,6938,-5202,0,294,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.185,0.030833,1.001900e+09,1.669833e+08,5048499.995,841416.665833,-214.0,0.002,0.002,99999.0,99999.0,-519.0,0.01,0.01,12.0,12.0,-5021.0,0.431564,-0.122432
52,105.0,15,18,7.46,27.1,NaN,7.54,4.09,NaN,NaN,3.2,NaN,NaN,18231.0,97.92,0.00,0.0,0.0,0.0,0.0,0.03,2.05,0.0,0.0,0.0,0.00,0.0,19,70,81,13,87,1,2,0.0100,0.01,0.0294,1.000,0.99,18048.0,2009-12-31,8966,9130,-3010,0,297,2,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.185,0.030833,1.001

In [114]:
summ_f = summ_f.sort_values(["CRSP_FUNDNO", "CALDAT"])

In [115]:
summ_f.shape

(137945, 149)

In [ ]:
### Run the model and identify any possible effects.

In [116]:
import numpy as np
#Keeping 15% for the validation set, thats about 1000 instances to test on
msk = np.random.rand(len(summ_f)) < 0.15
summ_f["is_test"] = msk

In [137]:
summ_f.to_hdf("E:DS\Funds\summ_f.h5", 'key_to_store', table=True, mode='w')

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import _pickle as pickle
from datetime import timedelta

summ_f = pd.read_hdf("E:DS\Funds\summ_f.h5")

In [2]:
summ_d = summ_f

In [49]:
summ_d.shape

(137945, 150)

In [52]:
summ_d = summ_d.drop_duplicates(["CRSP_FUNDNO","CALDAT"]);summ_d.shape

(137945, 150)

In [67]:
summ_d = summ_d.sort_values("CALDAT")

In [68]:
## About 7% report 0% return - this is curious
summ_d["annual_ret_f"].value_counts()[0]/len(summ_d)

0.06771539381637609

In [69]:
pd.options.display.max_columns = None

In [70]:
## Data 2002 - 2012 
summ_d.tail()

,CRSP_FUNDNO,MGMT_CD,MGR_NAME,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDAT,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,annual_ret_f,annual_ret,is_test
449296,59755.0,13,15,11.02,5.2,0.003031,11.70,10.33,0.03340,NaN,NaN,NaN,NaN,20392.0,88.31,0.0,0.0,0.00,0.00,0.00,0.00,1.00,0.0,0.0,0.0,10.69,0.00,19,70,81,13,87,1,2,0.0023,0.0025,0.0123,0.313,0.61,20331.0,2015-12-31,756,1094,-819,2191,93,224,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0.033400,0.033400,10.93,10.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.03340,0.03340,0.000000,0.000000,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,10.93,10.93,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0525,0.013125,1.000850e+09,2.502125e+08,12875.0,3218.750000,-1551.0,0.01,0.01,1.0,1.0,-1419.0,0.01,0.01,12.0,12.0,-1551.0,-0.005415,-0.040905,False
147987,16396.0,15,18,28.45,1329.2,0.018379,36.77,28.02,0.52287,NaN,NaN,-0.005476,19389.0,20392.0,99.78,0.0,0.0,0.00,0.00,0.00,0.00,0.06,0.0,0.0,0.0,0.16,0.00,21,47,53,9,62,1,2,-99.0000,-99.0000,0.0048,0.479,0.15,20331.0,2015-12-31,7233,2921,-819,2740,10,231,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0.522870,0.522870,28.02,28.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,0.52287,0.52287,0.000000,0.000000,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,28.02,28.02,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1900,0.027143,1.004100e+09,1.434429e+08,73250.0,10464.285714,-245.0,0.02,0.02,1.0,1.0,-5364.0,0.01,0.01,12.0,12.0,-518.0,-0.186911,0.070354,False
110584,12735.0,11,15,32.23,545.7,NaN,36.94,31.54,NaN,1.95424,NaN,NaN,NaN,20392.0,93.95,0.0,0.0,0.00,0.00,0.00,4.57,1.49,0.0,0.0,0.0,0.00,0.00,15,149,169,11,99,1,2,0.0075,0.0075,0.0174,0.641,0.18,20269.0,2015-12-31,5358,5477,-819,2283,13,286,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1.954240,0.977120,63.52,3

In [71]:
X = summ_d[summ_d["is_test"]==False]
X_test = summ_d[summ_d["is_test"]==True]

In [72]:
msk = np.random.rand(len(X)) < 0.15
X["is_valid"] = msk

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
X_train = X[X["is_valid"]==False]
X_valid = X[X["is_valid"]==True]

In [74]:
y_train = X_train["annual_ret_f"]
y_valid = X_valid["annual_ret_f"]

In [75]:
drop_cols = ["is_valid","is_test","annual_ret_f","CALDAT"]
drop_sans = ["is_test","annual_ret_f","CALDAT"]

In [76]:
X_full = X.drop(drop_cols,axis=1)
y_full = X["annual_ret_f"]

In [77]:
#del X_train["CALDAT"], X_valid["CALDAT"]

In [78]:
X_train = X_train.drop(drop_cols,axis=1)
X_valid = X_valid.drop(drop_cols,axis=1)

In [79]:
y_test = X_test["annual_ret_f"]
X_test = X_test.drop(drop_sans,axis=1)

In [82]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from lightgbm import LGBMRegressor

n_splits = 10
#cvv = KFold(n_splits=n_splits, random_state=42)

cvv = TimeSeriesSplit(n_splits=n_splits)
oof_preds = np.zeros(X_full.shape[0])

sub = y_test.to_frame()

sub["Target"] = 0

feature_importances = pd.DataFrame()
avg_iter = 0

for i, (fit_idx, val_idx) in enumerate(cvv.split(X_full, y_full)):

    X_fit = X_full.iloc[fit_idx]
    y_fit = y_full.iloc[fit_idx]
    X_val = X_full.iloc[val_idx]
    y_val = y_full.iloc[val_idx]
    print(X_full.shape)
    print(X_fit.shape)
    
    model = LGBMRegressor(
**params
    )

    model.fit(
        X_fit,
        y_fit,
        eval_set=[(X_fit, y_fit), (X_val, y_val)],
        eval_names=('fit', 'val'),
        eval_metric='mae',
        early_stopping_rounds=150,
        verbose=False
    )
    
    print("itteration: ", model.best_iteration_)
    avg_iter += model.best_iteration_
    oof_preds[val_idx] = model.predict(X_val, num_iteration=model.best_iteration_)
    sub['Target'] += model.predict(X_test, num_iteration=model.best_iteration_)
    
    fi = pd.DataFrame()
    fi["feature"] = X_full.columns
    fi["importance"] = model.feature_importances_
    fi["fold"] = (i+1)
    
    feature_importances = pd.concat([feature_importances, fi], axis=0)
    
    print("Fold {} MAE: {:.8f}".format(i+1, mean_absolute_error(y_val, oof_preds[val_idx])))

print('In Sample MAE score %.8f' % mean_absolute_error(y_full, oof_preds)) 
print('Out of sample MAE: ', mean_absolute_error(y_test, sub['Target']/n_splits))
print("avg iteration: ",(avg_iter/n_splits))


(117317, 147)
(10667, 147)
itteration:  100
Fold 1 MAE: 0.12042585
(117317, 147)
(21332, 147)
itteration:  100
Fold 2 MAE: 0.10485119
(117317, 147)
(31997, 147)
itteration:  100
Fold 3 MAE: 0.08302148
(117317, 147)
(42662, 147)
itteration:  99
Fold 4 MAE: 0.08864262
(117317, 147)
(53327, 147)
itteration:  100
Fold 5 MAE: 0.05836761
(117317, 147)
(63992, 147)
itteration:  98
Fold 6 MAE: 0.08331090
(117317, 147)
(74657, 147)
itteration:  100
Fold 7 MAE: 0.04393979
(117317, 147)
(85322, 147)
itteration:  100
Fold 8 MAE: 0.04377303
(117317, 147)
(95987, 147)
itteration:  100
Fold 9 MAE: 0.04515577
(117317, 147)
(106652, 147)
itteration:  100
Fold 10 MAE: 0.03517087
In Sample MAE score 0.07973325
Out of sample MAE:  0.06366064163609629
avg iteration:  99.7


In [94]:
fi = fi.sort_values("importance",ascending=False); fi.head(15)

,feature,importance,fold
7,NAV_52W_L,234,10
3,NAV_LATEST,180,10
44,CALDT_M_NAV_52W_L_DT,160,10
146,annual_ret,138,10
6,NAV_52W_H,135,10
45,CALDT_M_NAV_52W_H_DT,125,10
2,MGR_NAME,80,10
13,ASSET_DT,74,10
27,CRSP_OBJ_CD,67,10
4,TNA_LATEST,63,10


In [88]:
## Standard Benchmarks

y_mean = y_test.to_frame()
y_mean["annual_ret_f"] = y_train.mean()


y_median = y_test.to_frame()
y_median["annual_ret_f"] = y_train.median()

y_mode = y_test.to_frame()
y_mode["annual_ret_f"] = y_train.mode().values[0]


print("Train")
print("Mean: ",mean_absolute_error(y_test, y_mean),
      "Median: ",mean_absolute_error(y_test, y_median),
      "Mode: ",mean_absolute_error(y_test, y_mode),
      "Random",mean_absolute_error(y_test, y_train.sample(len(y_test)))) 

## Mean Test Distribution Peaking Naive Benchmark (TDPNB)
y_mean = y_test.to_frame()
y_mean["annual_ret_f"] = y_test.mean()


# Median TDBPN
y_median = y_test.to_frame()
y_median["annual_ret_f"] = y_test.median()

## Mode TDPNB
y_mode = y_test.to_frame()
y_mode["annual_ret_f"] = y_test.mode().values[0]
    
print("Test Peaking")
print("Mean: ",mean_absolute_error(y_test, y_mean),
      "Median: ",mean_absolute_error(y_test, y_median),
      "Mode: ",mean_absolute_error(y_test, y_mode),
      "Random",mean_absolute_error(y_test, y_test.sample(len(y_test))))

Train
Mean:  0.1632452218944914 Median:  0.1522414774704918 Mode:  0.15232371208317386 Random 0.22991562357010667
Test Peaking
Mean:  0.16426964922830112 Median:  0.1522399651885496 Mode:  0.15232371208317386 Random 0.23107880029567862


In [91]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse


def Standardisation(X_train, X_test, leave_out=None):
    from sklearn.preprocessing import StandardScaler
    if leave_out:
        leave = X_train[leave_out]
        X_train = X_train.drop(leave_out,axis=1)

        listed = list(X_train)
        scaler = StandardScaler()
        try:
            X_train = scaler.fit_transform(X_train)
        except:
            print("error_triggered")
            X_train = scaler.fit_transform(X_train.fillna(X_train.mean()))
            X_test = X_test.fillna(X_test.mean())
        
        X_test = scaler.transform(X_test)
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        X_train.columns = listed
        X_train = pd.concat((X_train,leave ),axis=1)
        X_test.columns = listed
        X_test = pd.concat((X_test,leave ),axis=1)
    else:
        scaler = StandardScaler()
        listed = list(X_train)
        try:
            X_train = scaler.fit_transform(X_train)
        except:
            print("error_triggered")
            X_train = scaler.fit_transform(X_train.fillna(X_train.mean()))
            X_test = X_test.fillna(X_test.mean())
        X_test = scaler.transform(X_test)
        X_test = pd.DataFrame(X_test)
        X_test.columns = listed
        
        X_train = pd.DataFrame(X_train)
        X_train.columns = listed
        
    return X_train, X_test

X_train_s, X_test_s = Standardisation(X_train,X_test,[])
_, X_valid_s = Standardisation(X_train,X_valid,[])


error_triggered
error_triggered


In [90]:
X_train_s.head()

,CRSP_FUNDNO,MGMT_CD,MGR_NAME,NAV_LATEST,TNA_LATEST,YIELD,NAV_52W_H,NAV_52W_L,DIV_YTD,CAP_GAINS_YTD,UNREALIZED_APP_DEP,MATURITY,MATURITY_DT,ASSET_DT,PER_COM,PER_PREF,PER_CONV,PER_CORP,PER_MUNI,PER_GOVT,PER_OTH,PER_CASH,PER_BOND,PER_ABS,PER_MBS,PER_EQ_OTH,PER_FI_OTH,CRSP_OBJ_CD,LIPPER_CLASS,LIPPER_CLASS_NAME,LIPPER_OBJ_CD,LIPPER_OBJ_NAME,LIPPER_ASSET_CD,LIPPER_TAX_CD,ACTUAL_12B1,MAX_12B1,EXP_RATIO,MGMT_FEE,TURN_RATIO,FISCAL_YEAREND,CALDT_M_FIRST_OFFER_DT,CALDT_M_MGR_DT,CALDT_M_END_DT,CALDT_M_UNREALIZED_APP_DT,CALDT_M_NAV_52W_L_DT,CALDT_M_NAV_52W_H_DT,CALDT_M_TNA_LATEST_DT,CALDT_M_NAV_LATEST_DT,OPEN_TO_INV_EMPT,OPEN_TO_INV_N,OPEN_TO_INV_Y,SALES_RESTRICT_EMPT,SALES_RESTRICT_N,SALES_RESTRICT_Y,INDEX_FUND_FLAG_B,INDEX_FUND_FLAG_D,INDEX_FUND_FLAG_E,INDEX_FUND_FLAG_EMPT,ACCRUAL_FUND_EMPT,ACCRUAL_FUND_N,ACCRUAL_FUND_Y,RETAIL_FUND_EMPT,RETAIL_FUND_N,RETAIL_FUND_Y,INST_FUND_EMPT,INST_FUND_N,INST_FUND_Y,M_FUND_EMPT,M_FUND_N,VAU_FUND_EMPT,VAU_FUND_N,VAU_FUND_Y,ET_FLAG_EMPT,ET_FLAG_F,ET_FLAG_N,DIS_AMT_sum,DIS_AMT_mean,REINVEST_NAV_sum,REINVEST_NAV_mean,SPL_RATIO_sum,SPL_RATIO_mean,CL_sum,CL_mean,CS_sum,CS_mean,D_sum,D_mean,DT_sum,DT_mean,DW_sum,DW_mean,OTHER_sum,OTHER_mean,DIS_AMT_x_CL_sum,DIS_AMT_x_CL_mean,DIS_AMT_x_CS_sum,DIS_AMT_x_CS_mean,DIS_AMT_x_D_sum,DIS_AMT_x_D_mean,DIS_AMT_x_DT_sum,DIS_AMT_x_DT_mean,DIS_AMT_x_DW_sum,DIS_AMT_x_DW_mean,DIS_AMT_x_OTHER_sum,DIS_AMT_x_OTHER_mean,REINVEST_NAV_x_CL_sum,REINVEST_NAV_x_CL_mean,REINVEST_NAV_x_CS_sum,REINVEST_NAV_x_CS_mean,REINVEST_NAV_x_D_sum,REINVEST_NAV_x_D_mean,REINVEST_NAV_x_DT_sum,REINVEST_NAV_x_DT_mean,REINVEST_NAV_x_DW_sum,REINVEST_NAV_x_DW_mean,REINVEST_NAV_x_OTHER_sum,REINVEST_NAV_x_OTHER_mean,SPL_RATIO_x_CL_sum,SPL_RATIO_x_CL_mean,SPL_RATIO_x_CS_sum,SPL_RATIO_x_CS_mean,SPL_RATIO_x_D_sum,SPL_RATIO_x_D_mean,SPL_RATIO_x_DT_sum,SPL_RATIO_x_DT_mean,SPL_RATIO_x_DW_sum,SPL_RATIO_x_DW_mean,SPL_RATIO_x_OTHER_sum,SPL_RATIO_x_OTHER_mean,FRONT_LOAD_sum,FRONT_LOAD_mean,DOLLAR_AMT_sum,DOLLAR_AMT_mean,front_loaded_sum,front_loaded_mean,DIFF_DATES_x,REAR_LOAD_sum,REAR_LOAD_mean,TIME_PERIOD_sum,TIME_PERIOD_mean,DIFF_DATES_y,REAR_LOAD_sum_c,REAR_LOAD_mean_c,TIME_PERIOD_sum_c,TIME_PERIOD_mean_c,DIFF_DATES,annual_ret
0,-1.697258,0.580388,0.116124,-0.490468,-0.174920,6.606159e-15,-0.374401,-0.486466,3.024302e-15,-1.941562e-15,6.265738e-17,-2.092910e-15,0.0,0.000000,-1.229296e-14,9.180175e-16,-1.102977e-15,3.875503e-15,4.901926e-17,3.949439e-15,-1.255621e-15,3.924389e-15,2.735558e-15,-5.236581e-16,2.409246e-15,1.465029e-16,5.976448e-16,-0.406125,-1.057775,-0.925520,-0.718122,-0.225232,-1.683268,-3.709868,0.946003,-1.501032e+00,0.091233,0.038278,0.300446,-1.794480,0.032098,2.226732,-2.461945,-0.676823,-0.745815,1.929031,-0.051938,-0.013645,-0.040229,-0.22834,0.23223,-0.107386,0.12045,-0.053932,-0.166121,-0.233738,-0.103197,0.312939,-0.107386,0.546821,-0.529204,-0.040229,-0.952629,0.955719,-0.040229,0.788436,-0.785744,-0.24455,0.24455,-0.036282,0.346785,-0.344402,0.191388,-0.179577,-0.06412,-7.769112e-15,-6.274067e-15,-6.052935e-16,2.227800e-16,-3.028188e-18,-3.909551e-18,-1.289136e-16,9.877609e-14,0.000000,-1.067712e-13,0.0,0.0,0.000000,-2.242775e-13,0.000000,8.961926e-15,-3.658088e-17,2.332985e-15,1.866333e-15,1.077553e-16,-1.279619e-16,9.968487e-16,0.0,0.0,1.063581e-15,1.713766e-15,-9.033432e-15,-5.514215e-15,-6.199892e-16,-1.360572e-16,-4.405749e-15,-2.606979e-15,-7.626138e-16,3.709233e-16,0.0,0.0,-4.047965e-15,-1.971186e-15,9.952406e-16,-2.462035e-15,1.598620e-16,5.274266e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.028188e-18,-3.909551e-18,0.593304,0.614280,0.027194,-0.412424,0.939351,0.036528,1.348705,-3.017798,-3.022849,9.210195,9.231338,0.945909,-0.603642,-0.607063,-0.061372,-0.019794,-1.379095,-1.583579
1,-0.064176,0.580388,0.116124,-0.262406,-0.165417,-2.280119e-01,-0.247970,-0.201662,-3.057856e-01,-1.941562e-15,6.265738e-17,-2.092910e-15,0.0,0.000000,-1.229296e-14,9.180175e-16,-1.102977e-15,3.875503e-15,4.901926e-17,3.949439e-15,-1.255621e-15,3.924389e-15,2.735558e-15,-5.236581e-16,2.409246

In [103]:
## Good Bench - LightGBM seems to perform better
from sklearn.linear_model import ElasticNet

svr = ElasticNet(alpha=0.02,l1_ratio=0.30,max_iter=20000).fit(X_train_s, y_train)
y_svr = svr.predict(X_test_s)
mean_absolute_error(y_test, y_svr)

0.09078413247143573